<a href="https://colab.research.google.com/github/RalfH1388/genai-lecture/blob/main/genai_rag.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Retrieval Augmented Generation (RAG)
# ---
# In diesem Beispiel erstellen wir ein RAG-System.
# Warum ist RAG sinnvoll? Weil ein LLM nicht per se inhaltlich korrekten Inhalt
# produziert, sondern wie ein "stochastischer Papagei" entlang seiner Trainings-
# daten Tokens basierend auf vorherigen Tokens produziert.
# Wenn wir uns sicher sein wollen, dass die Inhalte korrekt sind, brauchen wir
# also andere Methoden. Und manchmal ist es ohnehin so, dass wir sehr spezielle
# Informationen verarbeiten müssen (z.B. geheime firmeninterne Dokumente), von
# denen das LLM sowieso nicht direkt etwas Bescheid weiß.
# Die Grundidee eines RAG-Systems ist, dass ein Sprachmodell bei der
# Beantwortung von Fragen externe Wissensquellen durchsucht und relevante
# Informationen in seine Antwort integriert.

In [2]:
# Wir wollen hier mit den LLMs von OpenAI arbeiten. Da wir ein Backend-System
# entwickeln, hilft uns die UI-Version von ChatGPT nicht weiter, sondern wir
# brauchen Zugriff zur Developer-API mittels API Key. Jeder von Euch hat von mir
# entweder einen API Key von meinem persönlichen Account bekommen, oder nutzt
# einen aus seinem eigenen Account. Dieser Key muss - falls Ihr in Google
# Colab bleibt - als Secret hier hinterlegt werden, ansonsten in einem lokalen
# Environment. In Google Colab müsst Ihr links auf das Schlüssel-Symbol klicken
# und den Key dort copy pasten sowie dem Key einen Namen geben. Diesen Namen
# nutzt Ihr dann hier in der Klammer:

from google.colab import userdata
OPENAI_API_KEY = userdata.get('apikey_rh')

In [ ]:
!pip install langchain_openai
#!pip install openai
#!pip install langchain-core

In [6]:
# Nun nutzen wir ein LLM von OpenAI, indem wir die API direkt ansprechen, in
# diesem Fall das gpt-4o-mini:
from langchain_openai.chat_models import ChatOpenAI

model = ChatOpenAI(openai_api_key=OPENAI_API_KEY, model="gpt-4o-mini")

In [7]:
# Nun kann mit model.invoke() ein Prompt an die API geschickt werden, und das
# in model eingestellte Modell liefert die Antwort:
model.invoke("Was ist der Sinn des Lebens? Bitte fasse Dich kurz!")

AIMessage(content='Der Sinn des Lebens ist eine komplexe und individuelle Frage, die viele Philosophien, Religionen und Wissenschaften unterschiedlich beantworten. Einige mögliche Perspektiven sind:\n\n1. **Philosophische Ansätze**: Philosophen wie Sokrates, Nietzsche oder Sartre haben verschiedene Sichtweisen entwickelt, die von der Suche nach Wissen, über die Schaffung eigener Werte bis hin zur Akzeptanz von Absurdität reichen.\n\n2. **Religiöse Perspektiven**: Viele Religionen bieten Antworten, die oft auf dem Glauben an einen höheren Zweck oder an eine göttliche Ordnung basieren. Beispielsweise glauben Christen, dass der Sinn des Lebens darin besteht, Gott zu dienen und Nächstenliebe zu praktizieren, während Buddhisten das Streben nach Erleuchtung als zentral erachten.\n\n3. **Humanistische Sichtweisen**: Aus einer humanistischen Perspektive könnte der Sinn des Lebens darin gesehen werden, das Leben zu genießen, persönliche Beziehungen aufzubauen, das Wohl anderer zu fördern und ei

In [9]:
# Man kann hier noch viel mehr Dinge einstellen:
model2= ChatOpenAI(
    openai_api_key=OPENAI_API_KEY,
    model="gpt-4o-mini",
    temperature=0.7,           # Kreativität (0 = deterministisch, 1 = kreativ)
    max_tokens=100,            # Maximale Länge der Antwort
    top_p=1.0,                 # Nucleus Sampling (Standard: 1.0)
    frequency_penalty=0.0,     # Strafe für Wiederholungen
    presence_penalty=0.0,      # Motivation, neue Themen zu bringen
    request_timeout=30        # Timeout in Sekunden
)
model2.invoke("Was ist der Sinn des Lebens? Bitte fasse Dich kurz!")

AIMessage(content='Der Sinn des Lebens ist eine tiefgehende und oft philosophische Frage, die Menschen seit Jahrhunderten beschäftigt. Die Antworten darauf sind vielfältig und hängen stark von individuellen Überzeugungen, kulturellen Hintergründen und persönlichen Erfahrungen ab. \n\nEinige mögliche Perspektiven sind:\n\n1. **Persönliche Erfüllung**: Viele Menschen finden Sinn im Streben nach persönlichen Zielen, Leidenschaften und Hobbys. Dies kann die Verwirklichung von Träumen, das Erlernen neuer', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 100, 'prompt_tokens': 14, 'total_tokens': 114, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_34a54ae93c', 'id': 'chatcmpl-BrOwYFmhnWLD04NjBxULq0s8EA2x9', 'service_tier': 'def

In [10]:
# Man sieht jetzt im obigen Beispiel, dass etwas die Antwort kürzer geworden
# ist, da wir die max_tokens auf 100 begrenzt haben.

# Merke: oben haben wir den Code des Langchain OpenAI-Frameworks benutzt.
# Es gibt natürlich die Möglichkeit, auch die OpenAI Original SDK zu verwenden:

In [ ]:
!pip install openai

In [16]:
from openai import OpenAI

client = OpenAI(api_key=OPENAI_API_KEY, timeout=30)  # hier wird der Timeout gesetzt

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "user", "content": "Was ist der Sinn des Lebens? Bitte fasse Dich kurz!"}
    ],
    temperature=0.7,
    max_tokens=100,
    top_p=1.0,
    frequency_penalty=0.0,
    presence_penalty=0.0,
)

print(response.choices[0].message.content)

Der Sinn des Lebens ist eine tiefgründige Frage, die von verschiedenen Kulturen, Philosophien und Religionen unterschiedlich beantwortet wird. Manche Menschen finden Sinn im Streben nach Glück, in zwischenmenschlichen Beziehungen, in der Verwirklichung von Zielen oder in der Suche nach Wissen. Andere sehen den Sinn des Lebens in spirituellen oder religiösen Überzeugungen.

Philosophische Ansätze reichen von Existenzialismus, der den Einzelnen dazu ermutigt, seinen eigenen


In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

chain = model | parser
chain.invoke("Wie funktioniert ein Ottomotor? Bitte antworte in zwei bis drei Sätzen!")

'Ein Ottomotor funktioniert durch den Zyklus von Ansaugen, Verdichten, Arbeiten und Ausstoßen von Luft-Kraftstoff-Gemisch in den Zylindern. Während der Verdichtung wird das Gemisch komprimiert, und ein Zündfunke entzündet es, woraufhin die Explosion den Kolben nach unten drückt und mechanische Energie erzeugt. Anschließend wird die verbrannte Mischung durch den Auspuff ausgestoßen und der Zyklus beginnt von neuem.'

In [ ]:
from openai import OpenAI
client = OpenAI(api_key=OPENAI_API_KEY)

response = client.responses.create(
    model="gpt-4o-mini",
    input="Wie funktioniert ein Ottomotor? Bitte antworte in zwei bis drei Sätzen!"
)

print(response.output_text)

Ein Ottomotor funktioniert durch den vierstufigen Otto-Zyklus: Ansaugen, Verdichten, Arbeiten und Ausstoßen. Bei der Ansaugphase wird Luft und Kraftstoffgemisch in den Zylinder gezogen, anschließend wird es komprimiert und durch einen Zündfunken entzündet, wodurch eine explosionsartige Expansion entsteht, die den Kolben nach unten drückt. Schließlich wird die verbrannte Luft-Abluft-Mischung durch das Auslassventil nach außen geleitet.


'Ein Ottomotor funktioniert nach dem Viertaktprinzip, bestehend aus Ansaugen, Verdichten, Arbeits- und Ausstoßtakt. Während des Ansaugens wird ein Luft-Benzin-Gemisch in den Zylinder eingesogen, das dann im Verdichtungstakt durch den Kolben komprimiert wird. Die Zündung des Gemischs erfolgt durch einen Funken, wodurch der Kolben im Arbeitstakt nach unten gedrückt wird und der Zyklus mit dem Ausstoßtakt endet, in dem die verbrauchten Abgase ausgestoßen werden.'

In [ ]:
from langchain.prompts import ChatPromptTemplate

template = """
Beantworte die Frage basierend auf dem Kontext.
Wenn Du die Frage nicht beantworten kannst, antworte "Ich weiß es nicht".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt.format(context="Ralfs Bruder heißt Axl.", question="Wer ist Ralfs Bruder?")

'Human: \nBeantworte die Frage basierend auf dem Kontext.\nWenn Du die Frage nicht beantworten kannst, antworte "Ich weiß es nicht".\n\nContext: Ralfs Bruder heißt Axl.\n\nQuestion: Wer ist Ralfs Bruder?\n'

In [ ]:
chain = prompt | model | parser
chain.invoke({
    "context": "Ralfs Bruder heißt Axl",
    "question": "Wer ist Ralfs Bruder?"
})

'Ralfs Bruder ist Axl.'

In [ ]:
translation_prompt = ChatPromptTemplate.from_template(
    "Translate {answer} to {language}"
)

In [ ]:
from operator import itemgetter

translation_chain = (
    {"answer": chain, "language": itemgetter("language")} | translation_prompt | model | parser
)

translation_chain.invoke(
    {
        "context": "Ralfs Bruder heißt Axl. Er hat zwei weitere Geschwister.",
        "question": "Wie viele Geschwister hat Ralf?",
        "language": "English",
    }
)

"Ralf's brother Axl has two other siblings, so Ralf has a total of three siblings."

Let's read the transcription and display the first few characters to ensure everything works as expected.

In [ ]:
with open("data_interview.txt") as file:
    interview = file.read()

interview[:100]

"I think it's possible that physics has exploits and we should be trying to find them. arranging some"

In [ ]:
try:
    chain.invoke({
        "context": interview,
        "question": "Is reading papers a good idea?"
    })
except Exception as e:
    print(e)

In [ ]:
!pip install langchain-community

In [ ]:
from langchain_community.document_loaders import TextLoader

loader = TextLoader("interview.txt")
text_documents = loader.load()
text_documents

In [ ]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
text_splitter.split_documents(text_documents)[:5]

[Document(metadata={'source': 'interview.txt'}, page_content="I think it's possible that physics has exploits and we should be trying to find them. arranging some"),
 Document(metadata={'source': 'interview.txt'}, page_content='arranging some kind of a crazy quantum mechanical system that somehow gives you buffer overflow,'),
 Document(metadata={'source': 'interview.txt'}, page_content='buffer overflow, somehow gives you a rounding error in the floating point. Synthetic intelligences'),
 Document(metadata={'source': 'interview.txt'}, page_content="intelligences are kind of like the next stage of development. And I don't know where it leads to."),
 Document(metadata={'source': 'interview.txt'}, page_content='where it leads to. Like at some point, I suspect the universe is some kind of a puzzle. These')]

In [ ]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
documents = text_splitter.split_documents(text_documents)

In [ ]:
from langchain_openai.embeddings import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(openai_api_key=OPENAI_API_KEY)
embedded_query = embeddings.embed_query("Wer ist Ralfs Bruder?")

print(f"Embedding length: {len(embedded_query)}")
print(embedded_query[:10])

Embedding length: 1536
[0.01599571667611599, -0.019886909052729607, 0.01371423527598381, -0.013169214129447937, -0.027580568566918373, 0.014462053775787354, -0.013625510968267918, -0.0007193002384155989, 0.0008008948643691838, -0.01744065433740616]


In [ ]:
sentence1 = embeddings.embed_query("Ralfs Bruder ist Axl")
sentence2 = embeddings.embed_query("Michaelas Bruder ist Paul")

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

query_sentence1_similarity = cosine_similarity([embedded_query], [sentence1])[0][0]
query_sentence2_similarity = cosine_similarity([embedded_query], [sentence2])[0][0]

query_sentence1_similarity, query_sentence2_similarity

(np.float64(0.9093964737341363), np.float64(0.843505451391902))

In [ ]:
!pip install docarray

In [ ]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore1 = DocArrayInMemorySearch.from_texts(
    [
        "Ralfs Bruder heißt Axl",
        "Michaela und Paul sind Geschwister",
        "Dennis mag weiße Autos",
        "Anna Mutter ist Lehrerin",
        "Hektor fährt einen schwarzen Audi",
        "Michaela hat zwei Geschwister",
    ],
    embedding=embeddings,
)

/usr/local/lib/python3.11/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [ ]:
vectorstore1.similarity_search_with_score(query="Wer ist Ralfs Bruder?", k=3)

[(Document(metadata={}, page_content='Ralfs Bruder heißt Axl'),
  np.float64(0.9127635862951935)),
 (Document(metadata={}, page_content='Michaela und Paul sind Geschwister'),
  np.float64(0.8209651848203824)),
 (Document(metadata={}, page_content='Michaela hat zwei Geschwister'),
  np.float64(0.8025362350493098))]

In [ ]:
retriever1 = vectorstore1.as_retriever()
retriever1.invoke("Wer ist Ralfs Bruder?")

[Document(metadata={}, page_content='Ralfs Bruder heißt Axl'),
 Document(metadata={}, page_content='Michaela und Paul sind Geschwister'),
 Document(metadata={}, page_content='Michaela hat zwei Geschwister'),
 Document(metadata={}, page_content='Hektor fährt einen schwarzen Audi')]

In [ ]:
from langchain_core.runnables import RunnableParallel, RunnablePassthrough

setup = RunnableParallel(context=retriever1, question=RunnablePassthrough())
setup.invoke("Welche Farbe hat Hektors Auto?")

{'context': [Document(metadata={}, page_content='Hektor fährt einen schwarzen Audi'),
  Document(metadata={}, page_content='Dennis mag weiße Autos'),
  Document(metadata={}, page_content='Ralfs Bruder heißt Axl'),
  Document(metadata={}, page_content='Michaela hat zwei Geschwister')],
 'question': 'Welche Farbe hat Hektors Auto?'}

In [ ]:
chain = setup | prompt | model | parser
chain.invoke("Welche Farbe hat Hektors Auto?")

'Hektors Auto ist schwarz.'

In [ ]:
chain.invoke("Welches Auto fährt Hektor?")

'Hektor fährt einen schwarzen Audi.'

In [ ]:
vectorstore2 = DocArrayInMemorySearch.from_documents(documents, embeddings)

In [ ]:
chain = (
    {"context": vectorstore2.as_retriever(), "question": RunnablePassthrough()}
    | prompt
    | model
    | parser
)
chain.invoke("What is synthetic intelligence?")

'Synthetic intelligence refers to advanced artificial intelligence systems that are seen as the next stage of development in AI. These systems are thought to possess capabilities that allow them to uncover and solve complex problems or puzzles within the universe. Unlike traditional forms of AI, synthetic intelligences are anticipated to engage in tasks that involve understanding emotions, creativity, and generating art and ideas autonomously.'